In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：使用BigQuery为在线预测训练表格回归模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_regression_online_bq.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_regression_online_bq.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      查看在 GitHub 上的代码
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_regression_online_bq.ipynb">
      在 Google Cloud Notebooks 中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK创建表格回归模型，并使用Google Cloud的AutoML模型进行在线预测。您可以在[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)文档中找到更多信息。

### 数据集

本教程使用的数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)的GSOD数据集。您所使用的数据集版本仅使用年份、月份和日期字段来预测平均每日温度（mean_temp）的值。

### 目标

在本教程中，您将使用Vertex SDK从Python脚本中创建一个AutoML表格回归模型，并部署用于在线预测。您也可以使用`gcloud`命令行工具或在Cloud Console上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

费用

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/) 基于您的预期使用量生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Google Cloud笔记本，则您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。您需要以下内容：

- 云存储 SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简洁的指导：

1. [安装并初始化 SDK](https://cloud.google.com/sdk/docs/)。

2. [安装 Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装 virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用 Python 3 的虚拟环境。激活虚拟环境。

4. 要安装 Jupyter，请在终端中的命令行中运行 `pip3 install jupyter`。

5. 要启动 Jupyter，请在终端中的命令行中运行 `jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装最新版本的Python的Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，这样它才能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU运行时

此教程不需要GPU运行时。

### 设置您的Google Cloud项目

**无论您的笔记本环境如何，都需要以下步骤。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额度，用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、Compute Engine API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您将需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行视为shell命令，并会对以`$`为前缀的Python变量进行插值。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改“REGION”变量，该变量用于笔记本其余部分的操作。以下是Vertex AI支持的地区。我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不使用多区域存储桶来进行Vertex AI训练。并非所有地区都支持所有Vertex AI服务。

了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行直播教程会议，可能会使用共享的测试账户或项目。为了避免用户在创建的资源中发生名称冲突，您可以为每个实例会话创建一个时间戳，并将该时间戳附加到您在该教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，您的环境已经通过身份验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证，通过oAuth。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中，输入一个名称，然后点击**创建**。

在**将此服务帐户授予项目访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex”，并选择**Vertex管理员**。在过滤框中输入“Storage Object Admin”，并选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件将下载到本地环境。

在下面的单元格中，将您的服务帐户密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，都需要执行以下步骤。**

当您初始化用于Python的Vertex SDK时，您需要指定一个云存储暂存桶。这个暂存桶是在会话之间保存与您的数据集和模型资源相关的所有数据的地方。

请在下方设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时才能执行：运行以下单元格创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容来验证您对云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化Python的Vertex SDK

为您的项目和对应的存储桶初始化Python的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# 教程

现在您已经准备好开始创建自己的AutoML表格回归模型。

BigQuery培训数据的位置。

现在将变量`IMPORT_FILE`设置为BigQuery中数据表的位置。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"

### 创建数据集

接下来，使用`TabularDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `bq_source`：或者，从BigQuery表中导入数据项到`Dataset`资源。

此操作可能需要几分钟时间。

In [ ]:
dataset = aip.TabularDataset.create(
    display_name="NOAA historical weather data" + "_" + TIMESTAMP,
    bq_source=[IMPORT_FILE],
)

label_column = "mean_temp"

print(dataset.resource_name)

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "year"}},
    {"auto": {"column_name": "month"}},
    {"auto": {"column_name": "day"}},
]

label_column = "mean_temp"

### 创建并运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练管道，2) 运行该管道。

#### 创建训练管道

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练管道，并使用以下参数：

- `display_name`：用于`TrainingJob`资源的可读名称。
- `optimization_prediction_type`：训练模型的任务类型。
  - `classification`：表格分类模型。
  - `regression`：表格回归模型。
- `column_transformations`：（可选）要应用于输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化对象是训练管道的DAG（有向无环图）。

In [ ]:
dag = aip.AutoMLTabularTrainingJob(
    display_name="gsod_" + TIMESTAMP,
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=TRANSFORMATIONS,
)

print(dag)

#### 运行训练管道

接下来，您可以通过调用`run`方法来运行DAG以启动训练作业，参数如下：

- `dataset`：训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：用作标签的列名称。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，在服务认为无法进一步改善模型目标度量前，训练可能在使用整个预算之前完成。

完成`run`方法后将返回`Model`资源。

训练管道的执行将花费最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="gsod_" + TIMESTAMP,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

## 查看模型评估分数
在模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=gsod_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

## 部署模型

接下来，部署您的模型进行在线预测。要部署模型，您需要调用`deploy` 方法，并提供以下参数：

- `machine_type`：计算机类型。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

发送一个在线预测请求

向部署的模型发送一个在线预测。

制作测试项目

您将使用合成数据作为测试数据项目。不必担心我们正在使用合成数据 - 我们只是想演示如何进行预测。

In [ ]:
INSTANCE = {"year": "1932", "month": "11", "day": "6"}

### 进行预测

现在您的`Model`资源已部署到一个`Endpoint`资源，您可以通过将预测请求发送到`Endpoint`资源来进行在线预测。

#### 请求

每个实例的格式为：

    [feature_list]

由于`predict()`方法可以接受多个项（实例），请将您的单个测试项作为一个测试项列表发送。

#### 响应

从`predict()`调用中返回的响应是一个Python字典，包含以下条目：

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `value`: 每个预测的预测值。
- `deployed_model_id`: 进行预测的部署的`Model`资源的 Vertex AI 标识符。

In [ ]:
instances_list = [INSTANCE]

prediction = endpoint.predict(instances_list)
print(prediction)

### 下线模型

当预测工作完成后，您需要从`Endpoint`资源中下线模型。这将取消所有计算资源，并停止对部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理在此项目中使用的所有Google Cloud资源，可以删除您用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- AutoML训练任务
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME